In [ ]:
!pip install omegaconf

In [ ]:
!pip install hydra-core --upgrade

In [ ]:
!pip install regex

In [ ]:
!pip install sacremoses

In [ ]:
!pip install subword-nmt

In [ ]:
!pip install fastBPE

In [ ]:
!pip install Cython

In [1]:
import torch
import fairseq
import PySimpleGUI as sg
import tkinter

In [3]:
torch.__version__

'1.6.0+cu101'

In [2]:
# List available models
#torch.hub.list('pytorch/fairseq')  # [..., 'transformer.wmt16.en-de', ... ]

# Load a transformer trained on WMT'16 En-De
# Note: WMT'19 models use fastBPE instead of subword_nmt, see instructions below
#de2en = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.de-en', checkpoint_file='model1.pt:model2.pt:model3.pt:model4.pt', tokenizer='moses', bpe='fastbpe')
path = '/home/scutum/Desktop/wmt19.de-en.joined-dict.ensemble/'
de2en = torch.hub._load_local(path, 'transformer.wmt19.de-en', checkpoint_file='model1.pt:model2.pt:model3.pt:model4.pt', tokenizer='moses', bpe='fastbpe')

de2en.eval()  # disable dropout

# The underlying model is available under the *models* attribute
assert isinstance(de2en.models[0], fairseq.models.transformer.TransformerModel)

AttributeError: module 'torch.hub' has no attribute '_load_local'

In [8]:
# Move model to GPU for faster translation
de2en.cuda()

GeneratorHubInterface(
  (models): ModuleList(
    (0): TransformerModel(
      (encoder): TransformerEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(42024, 1024, padding_idx=1)
        (embed_positions): SinusoidalPositionalEmbedding()
        (layers): ModuleList(
          (0): TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (dropout_module): FairseqDropout()
            (activation_dropout_module): FairseqDropout()
            (fc1): Linear(in_feat

In [ ]:
# Translate a sentence
de2en.translate('Liebe ist Glück, aber auch Leid.')
# 'Hallo Welt!'

In [ ]:
# Batched translation
de2en.translate(['Hallo Welt!', 'Die Katze saß auf der Matte.'])
# ['Hallo Welt!', 'Die Katze saß auf der Matte.']

In [9]:
static_texts = [
    [
        sg.Text("German Sentence:", size=(30, 6), enable_events=True, pad=((0,0),(60,0))),
        sg.Multiline(size=(100, 6), key='-GERMAN-')
    ],
    [
        sg.Button('Translate', auto_size_button=True, size=(130,1), key='-TRANSLATE-')
    ],
    [
        sg.Text("Translated(English) Sentence:", size=(30, 6), enable_events=True, pad=((0,0),(60,0))),
        sg.Multiline(size=(100, 6), key='-ENGLISH-')
    ]
]


In [10]:
layout = [
    [
        sg.Column(static_texts)
    ]
]

In [11]:
window = sg.Window("FMEA Translator", layout)

In [12]:
d = {'x': 2}

d.update(x=3, y = 3, z = 0)
print(d)

{'x': 3, 'y': 3, 'z': 0}


In [ ]:
while True:
    event, values = window.read()
    if event == "Exit" or event == sg.WIN_CLOSED:
        break
    elif event == '-TRANSLATE-':
        print(values['-GERMAN-'])
        window.Element('-ENGLISH-').Update(de2en.translate(values['-GERMAN-']))
        #values.update({1: de2en.translate(values[0])})
        

Rex, der hungrigste Hund der Welt, verlor plötzlich seinen Appetit.

